In [2]:
import numpy as np
import pandas as pd


# =======================================================
# --- Functions ---
# =======================================================
def simulate_one_run_of_particle_system(
    gamma, beta, v, x0, T, K, N
):
    """
    Simulates ONE run of the N-particle system until time T
    and returns the value of the first particle X_T^{1,N,h}.

    Parameters:
    gamma (float): Parameter gamma in the SDE.
    beta (float): Parameter beta in the SDE.
    v (float): Parameter v in the SDE.
    x0 (float): Initial value for all particles.
    T (float): Terminal time.
    K (int): Number of time steps.
    N (int): Number of particles in the system.

    Returns:
    float: The value of the first particle X_T^{1,N,h} at terminal time T for this single run.
    """
    h = T / K  # Time step size
    X = np.full(N, x0) # Particle values for this single run

    for _ in range(K): # Time stepping
        dW = np.sqrt(h) * np.random.randn(N)
        mean_X = np.mean(X)
        drift_term = (gamma * X + beta * mean_X) * h
        diffusion_term = v * dW
        X = X + drift_term + diffusion_term
    
    return X[0] # Value of the first particle at time T


def estimate_moment_and_get_terminal_values(
    gamma, beta, v, x0, T, K, N, num_runs
):
    """
    Estimates E[X_T^{1,N,h}] by running the particle system `num_runs` times.
    Returns all terminal values of X_T^{1,N,h} for precision calculation.

    Parameters:
    (Same as simulate_one_run_of_particle_system plus num_runs)
    num_runs (int): Number of independent Monte Carlo simulations.

    Returns:
    numpy.ndarray: Array of shape (num_runs,) containing X_T^{1,N,h} from each run.
    """
    run_terminal_values_X1 = np.zeros(num_runs)

    for run_idx in range(num_runs):
        
        run_terminal_values_X1[run_idx] = simulate_one_run_of_particle_system(
            gamma, beta, v, x0, T, K, N
        )
    return run_terminal_values_X1


def calculate_first_moment(gamma, beta, x0, T, K):
    """
    Calculates the closed-form first moment of the discretized particle system.
    E[X_T^{1,N,h}] = (1 + (gamma+beta)h)^K * x0
    """
    h = T / K
    first_moment = (1 + (gamma + beta) * h)**K * x0
    return first_moment

def calculate_precision(values, num_runs):
    """
    Calculates the precision (half-width of 95% CI) for the estimated mean.
    """
    if num_runs <= 1:
        return np.nan
    variance_of_values = np.var(values, ddof=1) # ddof=1 for sample variance
    standard_error_of_mean = np.sqrt(variance_of_values / num_runs)
    precision = 1.96 * standard_error_of_mean
    return precision


# =======================================================
# --- Parameters ---
# =======================================================
# gamma = 1/2 # paper wrote
# beta = 4/5 # paper wrote
gamma = 0.1 # paper may used
beta = 0.2 # paper may used
v_squared = 1/2
v = np.sqrt(v_squared)
x0 = 1
T = 1
K = 50
num_runs = 5 * 10**5
N_values = [20, 40, 80, 160, 320]


# =======================================================
# --- Main Simulation Loop ---
# =======================================================
first_moment = calculate_first_moment(
    gamma, beta, x0, T, K
)
print(f"Closed-form discretized value for E[X_T^(1,N,h)]: {first_moment:.5f}\n")

table_data = []

for N in N_values:
    print(f"Running for N = {N} particles...")

    # Get all terminal values from num_runs simulations
    terminal_values_X1_for_N = estimate_moment_and_get_terminal_values(
        gamma, beta, v, x0, T, K, N, num_runs
    )

    estimated_moment = np.mean(terminal_values_X1_for_N)
    difference = estimated_moment - first_moment
    precision = calculate_precision(terminal_values_X1_for_N, num_runs)

    table_data.append({
        "Nb. particles": N,
        "Estimated first moment": estimated_moment,
        "Difference": difference,
        "Precision": precision
    })
    # print(f"Finished N = {N}. Estimated moment: {estimated_moment:.5f}, Difference: {difference:.5f}, Precision: {precision:.5f}\n")


# =======================================================
# --- Table Generation ---
# =======================================================
df_table1 = pd.DataFrame(table_data)
# df_table1_styled = df_table1.style.format({ # For Jupyter/IPython display
#     "Estimated first moment": "{:.5f}",
#     "Difference": "{:.5f}",
#     "Precision": "{:.5f}"
# })

print(f"Closed-form discretized value (reference for 'Difference'): {first_moment:.5f}")

# For cleaner printing of the dataframe in a terminal
desired_columns = ["Nb. particles", "Estimated first moment", "Difference", "Precision"]
df_print = df_table1[desired_columns].copy()
df_print["Estimated first moment"] = df_print["Estimated first moment"].map('{:.5f}'.format)
df_print["Difference"] = df_print["Difference"].map('{:.5f}'.format) # Adjust format if very small values
df_print["Precision"] = df_print["Precision"].map('{:.5f}'.format)
print(df_print.to_string(index=False))

Closed-form discretized value for E[X_T^(1,N,h)]: 1.34865

Running for N = 20 particles...
Running for N = 40 particles...
Running for N = 80 particles...
Running for N = 160 particles...
Running for N = 320 particles...
Closed-form discretized value (reference for 'Difference'): 1.34865
 Nb. particles Estimated first moment Difference Precision
            20                1.34821   -0.00044   0.00207
            40                1.35015    0.00150   0.00207
            80                1.34886    0.00021   0.00206
           160                1.34799   -0.00066   0.00206
           320                1.34642   -0.00223   0.00206
